# Experiments with Home Credit

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score

import credit_pipeline.data_exploration as dex
from credit_pipeline.training import *

% load_ext autoreload
% autoreload 2

UsageError: Line magic function `%` not found.


## Loading and Cleaning

In [2]:
seed_number = 0

In [3]:
path = "../data/HomeCredit/"
df = dex.read_csv_encoded(path, 'application_train.csv').sample(10000)

In [4]:
columns_to_drop = dex.check_missing(df, 50,  False)
columns_to_drop

['COMMONAREA_MODE',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_MEDI',
 'FONDKAPREMONT_MODE',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAPARTMENTS_MODE',
 'FLOORSMIN_MODE',
 'FLOORSMIN_AVG',
 'FLOORSMIN_MEDI',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE',
 'YEARS_BUILD_AVG',
 'OWN_CAR_AGE',
 'LANDAREA_MODE',
 'LANDAREA_MEDI',
 'LANDAREA_AVG',
 'BASEMENTAREA_AVG',
 'BASEMENTAREA_MODE',
 'BASEMENTAREA_MEDI',
 'EXT_SOURCE_1',
 'NONLIVINGAREA_MEDI',
 'NONLIVINGAREA_AVG',
 'NONLIVINGAREA_MODE',
 'ELEVATORS_MODE',
 'ELEVATORS_MEDI',
 'ELEVATORS_AVG',
 'APARTMENTS_MODE',
 'APARTMENTS_MEDI',
 'APARTMENTS_AVG',
 'WALLSMATERIAL_MODE',
 'ENTRANCES_AVG',
 'ENTRANCES_MODE',
 'ENTRANCES_MEDI',
 'HOUSETYPE_MODE',
 'LIVINGAREA_AVG',
 'LIVINGAREA_MODE',
 'LIVINGAREA_MEDI',
 'FLOORSMAX_MODE',
 'FLOORSMAX_AVG',
 'FLOORSMAX_MEDI']

In [5]:
df = df.drop(columns_to_drop, axis=1)

In [6]:
df_cols = df.columns.to_list()
obj_cols = dex.list_by_type(df, ['O'])
obj_cols

['NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'EMERGENCYSTATE_MODE']

In [7]:
def days_to_years(dataframe, col_name):
        """
        Converts values from string to numeric.
        Uses the map function to convert the information on days employed to years employed
        """
        df_name = dataframe.copy()

        if col_name in df.columns:
            #Converts values from string to numeric.
            df_name[col_name] = pd.to_numeric(df_name[col_name], errors='coerce')

            #drops null values on the column
            df_name = df_name.dropna(subset=[col_name])

            #Use the map function to convert the information on days employed to years employed
            year = df_name.loc[:, col_name].map(lambda x: int(abs(x / 365)), na_action=None)
            df_name['YEARS'+col_name[4:]] = year

            #drops the column
            df_name = df_name.drop(col_name, axis=1)

        return df_name


df = days_to_years(df, "DAYS_EMPLOYED")
df = days_to_years(df, 'DAYS_BIRTH')

## Training Basic Models

In [8]:
X_acp = df.iloc[:, (df.columns != "TARGET") & (df.columns != "SK_ID_CURR")]
y_acp = df["TARGET"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X_acp, 
    y_acp, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_acp
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, 
    y_train, 
    test_size = 0.2, 
    random_state=seed_number, 
    stratify=y_train
)

In [10]:
classifiers = {
    "Logistic Regression": LogisticRegression,
    "Random Forest": RandomForestClassifier,
    "LightGBM": LGBMClassifier
}

In [24]:
param_spaces = {
    "LogisticRegression": {
        'C': {'low': 0.001, 'high': 10, 'log': True, 'type':'float'},
        'max_iter': {'low': 1000, 'high': 1000, 'step':1, 'type':'int'},
        'penalty': {'choices': ['l2'], 'type':'categorical'}
    },
    "RandomForestClassifier": {
        'n_estimators': {'low':10, 'high':150, 'step':1, 'type':'int'},
        'max_depth': {'low':2, 'high':10, 'type':'int'},
        'criterion': {'choices':['gini', 'entropy'], 'type':'categorical'}
    },
    "LGBMClassifier": {
        'learning_rate': {'low': 0.01, 'high': 1.0, 'type': 'float', 'log': True},
        'num_leaves': {'low': 2, 'high': 15, 'step': 1, 'type': 'int'},
        'max_depth': {'low': 1, 'high': 10, 'type': 'int'},
        'min_child_samples': {'low': 1, 'high': 50, 'step': 1, 'type': 'int'},
        'subsample': {'low': 0.1, 'high': 1.0, 'type': 'float'},
        'colsample_bytree': {'low': 0.1, 'high': 1.0, 'type': 'float'},
        'reg_alpha': {'low': 0.0, 'high': 0.5, 'type': 'float'},
        'reg_lambda': {'low': 0.0, 'high': 1.0, 'type': 'float'},
        # 'n_estimators': {'low': 50, 'high': 200, 'step': 1, 'type': 'int'},
        #'verbose': {'low':-1, 'high': -1, 'type': 'int'}
    }
}

In [28]:
study_logistic = optimize_model(LogisticRegression, param_spaces["LogisticRegression"], X_train, y_train, X_val , y_val, n_trials=100)
study_rf = optimize_model(RandomForestClassifier, param_spaces["RandomForestClassifier"], X_train, y_train, X_val , y_val, n_trials=100)
study_lgbm = optimize_model(LGBMClassifier, param_spaces["LGBMClassifier"], X_train, y_train, X_val , y_val, n_trials=100)

[I 2023-09-27 18:54:30,305] A new study created in memory with name: no-name-018f735a-3d08-4386-965d-fe3e843c0c3e
[I 2023-09-27 18:54:31,400] Trial 0 finished with value: 0.7420801492483765 and parameters: {'C': 0.15676677195506075, 'max_iter': 1000, 'penalty': 'l2'}. Best is trial 0 with value: 0.7420801492483765.
[I 2023-09-27 18:54:31,721] Trial 1 finished with value: 0.742961698725341 and parameters: {'C': 0.7257005721594281, 'max_iter': 1000, 'penalty': 'l2'}. Best is trial 1 with value: 0.742961698725341.
[I 2023-09-27 18:54:31,955] Trial 2 finished with value: 0.7425824274387399 and parameters: {'C': 0.25766385746135895, 'max_iter': 1000, 'penalty': 'l2'}. Best is trial 1 with value: 0.742961698725341.
[I 2023-09-27 18:54:32,287] Trial 3 finished with value: 0.7419571423446141 and parameters: {'C': 0.15119336467641012, 'max_iter': 1000, 'penalty': 'l2'}. Best is trial 1 with value: 0.742961698725341.
[I 2023-09-27 18:54:32,527] Trial 4 finished with value: 0.7399172778572196 and

[LightGBM] [Info] Number of positive: 532, number of negative: 5868
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001825 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3121
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083125 -> initscore=-2.400626
[LightGBM] [Info] Start training from score -2.400626


[I 2023-09-27 18:55:42,479] Trial 1 finished with value: 0.6249058228393068 and parameters: {'learning_rate': 0.8459126528049372, 'num_leaves': 7, 'max_depth': 8, 'min_child_samples': 27, 'subsample': 0.6112401049845391, 'colsample_bytree': 0.933036974463395, 'reg_alpha': 0.03551802909894347, 'reg_lambda': 0.08712929970154071}. Best is trial 0 with value: 0.7120767153056465.


[LightGBM] [Info] Number of positive: 532, number of negative: 5868
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000799 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3121
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083125 -> initscore=-2.400626
[LightGBM] [Info] Start training from score -2.400626


[I 2023-09-27 18:55:42,814] Trial 2 finished with value: 0.7292387410243399 and parameters: {'learning_rate': 0.01097581541938016, 'num_leaves': 13, 'max_depth': 8, 'min_child_samples': 44, 'subsample': 0.9807565080094875, 'colsample_bytree': 0.8192427077950513, 'reg_alpha': 0.23073968112646592, 'reg_lambda': 0.7805291762864555}. Best is trial 2 with value: 0.7292387410243399.


...

[LightGBM] [Info] Number of positive: 532, number of negative: 5868
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000815 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3121
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083125 -> initscore=-2.400626
[LightGBM] [Info] Start training from score -2.400626


[I 2023-09-27 18:56:12,492] Trial 98 finished with value: 0.7368933581397257 and parameters: {'learning_rate': 0.08315670928771703, 'num_leaves': 4, 'max_depth': 4, 'min_child_samples': 32, 'subsample': 0.8139519027815787, 'colsample_bytree': 0.5345661129792113, 'reg_alpha': 0.24076130364399467, 'reg_lambda': 0.3000678628293135}. Best is trial 18 with value: 0.7425055481238885.


[LightGBM] [Info] Number of positive: 532, number of negative: 5868
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000753 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3121
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083125 -> initscore=-2.400626
[LightGBM] [Info] Start training from score -2.400626


[I 2023-09-27 18:56:12,828] Trial 99 finished with value: 0.7387640881344466 and parameters: {'learning_rate': 0.05976475733878537, 'num_leaves': 5, 'max_depth': 5, 'min_child_samples': 45, 'subsample': 0.7370298386760102, 'colsample_bytree': 0.5888946602243817, 'reg_alpha': 0.2648582222281249, 'reg_lambda': 0.32408093121023573}. Best is trial 18 with value: 0.7425055481238885.


[LightGBM] [Info] Number of positive: 532, number of negative: 5868
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000699 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3113
[LightGBM] [Info] Number of data points in the train set: 6400, number of used features: 63
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.083125 -> initscore=-2.400626
[LightGBM] [Info] Start training from score -2.400626


In [29]:
print("Score for Logistic Regression: ", study_logistic.best_value)
print("Score for Random Forest: ", study_rf.best_value)
print("Score for LightGBM: ", study_lgbm.best_value)

Score for Logistic Regression:  0.7433050929983446
Score for Random Forest:  0.7396302617484406
Score for LightGBM:  0.7425055481238885


In [ ]:
def get_metrics(name_model_dict, X, y, threshold = 0.5):
    models_dict = {}
    for name, model in name_model_dict.items():
        if type(model) == list:
            y_prob = model[0].predict_proba(X)[:,1]
            threshold_model = model[1]
            y_pred = (y_prob >= threshold_model).astype('int')
        else:
            y_prob = model.predict_proba(X)[:,1]
            y_pred = (y_prob >= threshold).astype('int')

        models_dict[name] = (y_pred, y_prob)

    def get_metrics_df(models_dict, y_true,):
        metrics_dict = {
            "Overall AUC": (
                lambda x: roc_auc_score(y_true, x), False),
            " ------": (lambda x: "", True),
            "Balanced Accuracy": (
                lambda x: balanced_accuracy_score(y_true, x), True),
            "Accuracy": (
                lambda x: accuracy_score(y_true, x), True),
            "Precision": (
                lambda x: precision_score(y_true, x, zero_division=0), True),
            "Recall": (
                lambda x: recall_score(y_true, x), True),
            "F1": (
                lambda x: f1_score(y_true, x), True),
        }
        df_dict = {}
        for metric_name, (metric_func, use_preds) in metrics_dict.items():
            df_dict[metric_name] = [metric_func(preds) if use_preds else metric_func(scores)
                                    for model_name, (preds, scores) in models_dict.items()]
        return pd.DataFrame.from_dict(df_dict, orient="index", columns=models_dict.keys())

    return get_metrics_df(models_dict, y)

**TODO**

*   Add easy way to include default parameters in optuna
*   Improve create_pipeline function
*   Integrate pipeline package
*   Search hyperparameters range for classifiers
*   Warnings on pipeline given numeric unique number of values
